## Equity Earnings Projections
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/

### How to avoid getting blocked
- https://www.scraperapi.com/blog/5-tips-for-web-scraping 
- https://www.scrapehero.com/how-to-prevent-getting-blacklisted-while-scraping/

### Notes / Findings
- Based on cell below the 3 pastes of the same cell, Xpath is the best way to go.


### Notes/findings

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# https://hackernoon.com/scraping-yahoo-finance-data-using-python-ayu3zyl    

names=[]
prices=[]
changes=[]
percentChanges=[]
marketCaps=[]
totalVolumes=[]
circulatingSupplys=[]
 
CryptoCurrenciesUrl = "https://in.finance.yahoo.com/currencies"
r= requests.get(CryptoCurrenciesUrl)
data=r.text
soup=BeautifulSoup(data, "lxml")
 
# counter = 40
for i in range(40, 404, 14):
   for listing in soup.find_all('tr', attrs={'data-reactid':i}):
      for name in listing.find_all('td', attrs={'data-reactid':i+3}):
         names.append(name.text)
      for price in listing.find_all('td', attrs={'data-reactid':i+4}):
         prices.append(price.text)
      for change in listing.find_all('td', attrs={'data-reactid':i+5}):
         changes.append(change.text)
      for percentChange in listing.find_all('td', attrs={'data-reactid':i+7}):
         percentChanges.append(percentChange.text)
pd.DataFrame({"Names": names, "Prices": prices, "Change": changes, "% Change": percentChanges})

,Names,Prices,Change,% Change
0,USD/INR,71.5760,-0.2930,-0.4077%
1,EUR/INR,77.9504,-0.2676,-0.34%
2,GBP/INR,92.4631,-0.9559,-1.02%
3,AED/INR,19.4440,-0.0800,-0.41%
4,INR/JPY,1.5389,+0.0085,+0.56%
5,SGD/INR,51.2510,-0.1390,-0.27%
6,USD/IDR,"13,860.0000",0.0000,0.00%
7,USD/THB,31.8250,+0.1040,+0.33%
8,USD/MYR,4.2230,-0.0090,-0.21%
9,USD/ZAR,15.3145,+0.0930,+0.61%


In [ ]:
# https://hackernoon.com/scraping-yahoo-finance-data-using-python-ayu3zyl    

names=[]
prices=[]
changes=[]
percentChanges=[]
marketCaps=[]
totalVolumes=[]
circulatingSupplys=[]
ticker = 'BA'
 
analysis_url = "https://in.finance.yahoo.com/quote/CSII/analysis?p=' + 'ticker'
r= requests.get(analysis_url)
data=r.text
soup=BeautifulSoup(data, "lxml")
 
# counter = 40
for i in range(262, 404, 11):
   for listing in soup.find_all('tr', attrs={'data-reactid':i}):
      for name in listing.find_all('td', attrs={'data-reactid':i+3}):
         names.append(name.text)
      for price in listing.find_all('td', attrs={'data-reactid':i+4}):
         prices.append(price.text)
      for change in listing.find_all('td', attrs={'data-reactid':i+5}):
         changes.append(change.text)
      for percentChange in listing.find_all('td', attrs={'data-reactid':i+7}):
         percentChanges.append(percentChange.text)
pd.DataFrame({"Names": names, "Prices": prices, "Change": changes, "% Change": percentChanges})

In [ ]:
# Three different pastes, same cell: Current Estimate, Current Quarter:
# https://finance.yahoo.com/quote/CSCO/analysis?p=CSCO

# OuterHTML
<span class="Trsdu(0.3s) ">0.8</span>

# Selector
#Col1-0-AnalystLeafPage-Proxy > section > table:nth-child(5) > tbody > tr:nth-child(1) > td:nth-child(2) > span

# XPath
//*[@id="Col1-0-AnalystLeafPage-Proxy"]/section/table[4]/tbody/tr[1]/td[2]/span


In [7]:
# EPS Trend:
<span data-reactid="240">EPS Trend</span>

# Current Qtr.
<span data-reactid="243">Current Qtr.</span>
# Current Qtr. (Mar 2020)		
<span data-reactid="242"><span data-reactid="243">Current Qtr.</span><!-- react-text: 244 --> (<!-- /react-text --><span data-reactid="245">Mar</span><!-- react-text: 246 --> 2020)<!-- /react-text --></span>

# Current Year
<span data-reactid="255">Current Year</span>
# Current Year (2020)	
<span data-reactid="254"><span data-reactid="255">Current Year</span><!-- react-text: 256 --> (2020)<!-- /react-text --></span>

# Next Year
<span data-reactid="259">Next Year</span>
# Next Year (2021)
<span data-reactid="258"><span data-reactid="259">Next Year</span><!-- react-text: 260 --> (2021)<!-- /react-text --></span>
    
# Current Estimate
<span data-reactid="264">Current Estimate</span>

# Current estimates left to right:
<span data-reactid="264">Current Estimate</span>
<span class="Trsdu(0.3s) " data-reactid="266">-0.61</span>
<span class="Trsdu(0.3s) " data-reactid="268">0.31</span>
<span class="Trsdu(0.3s) " data-reactid="270">-5.72</span>
<span class="Trsdu(0.3s) " data-reactid="272">-16.56</span>

# 7 Days ago left to right:
<span data-reactid="275">7 Days Ago</span>
<span class="Trsdu(0.3s) " data-reactid="277">1.55</span>
<span class="Trsdu(0.3s) " data-reactid="279">1.55</span>
<span class="Trsdu(0.3s) " data-reactid="281">1.55</span>
<span class="Trsdu(0.3s) " data-reactid="283">1.55</span>


# 30 Days ago
<span data-reactid="286">30 Days Ago</span>
<span class="Trsdu(0.3s) " data-reactid="288">2.75</span>
<span class="Trsdu(0.3s) " data-reactid="290">2.75</span>
<span class="Trsdu(0.3s) " data-reactid="292">2.75</span>
<span class="Trsdu(0.3s) " data-reactid="294">2.75</span>

# 60 Days ago
<span data-reactid="297">60 Days Ago</span>
<span class="Trsdu(0.3s) " data-reactid="299">3.92</span>
<span class="Trsdu(0.3s) " data-reactid="301">3.92</span>
<span class="Trsdu(0.3s) " data-reactid="303">3.92</span>
<span class="Trsdu(0.3s) " data-reactid="305">3.92</span>

# 90 Days ago
<span data-reactid="308">90 Days Ago</span>
<span class="Trsdu(0.3s) " data-reactid="310">4.15</span>
<span class="Trsdu(0.3s) " data-reactid="312">4.15</span>
<span class="Trsdu(0.3s) " data-reactid="314">4.15</span>
<span class="Trsdu(0.3s) " data-reactid="316">4.15</span>








SyntaxError: invalid syntax (<ipython-input-7-5626f044940e>, line 2)

In [6]:
# Dow stocks
dow = [
    'MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'XOM', 'GS',
    'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV',
    'UTX', 'UNH', 'VZ', 'V', 'WMT', 'WBA'
    ] 




https://www.ryansmccoy.com/2019/01/28/download-eps-sales-estimates-using-python/
    

In [3]:
# https://www.ryansmccoy.com/2019/01/28/download-eps-sales-estimates-using-python/

import pandas as pd
from datetime import datetime

df_calendar = pd.read_html(r'https://finance.yahoo.com/calendar/earnings/')

df_all_est = []

for ticker in df_calendar[0]['Symbol'].tolist():
    try:
        print(f'Downloading Estimates for {ticker}')

        df_est = pd.read_html(r'https://finance.yahoo.com/quote/{ticker}/analysis?p={ticker}')
        eps_est, rev_est = df_est[0], df_est[1]

        rev_est = rev_est.melt(id_vars=['Revenue Estimate'],var_name='Period')
        rev_est['Ticker'] = ticker
        rev_est['Item'] = "REVENUES"
        rev_est.rename(index=str, columns={"Revenue Estimate": "Statistic"}, inplace=True)
        rev_est['DateTime'] = str(datetime.utcnow())
        df_all_est.append(rev_est)

        eps_est = eps_est.melt(id_vars=['Earnings Estimate'],var_name='Period')
        eps_est['Ticker'] = ticker
        eps_est['Item'] = "EPS"
        eps_est.rename(index=str, columns={"Earnings Estimate": "Statistic"}, inplace=True)
        eps_est['DateTime'] = str(datetime.utcnow())
        df_all_est.append(eps_est)

        print(eps_est.append(rev_est).to_string())

    except Exception as e:
        print(f"Error Downloading {ticker} \n{e}\n")

df_all_merged = pd.concat(df_all_est)
df_all_merged.reset_index(drop=True, inplace=True)
df_all_merged = df_all_merged.T.reindex(['DateTime', 'Ticker', 'Item', 'Statistic', 'Period', 'value']).T

df_all_merged.to_csv(r'{}_estimates.csv'.format(datetime.today().strftime("%Y%M%d")))

ValueError: No tables found